In [ ]:
# !sudo apt install swig

In [ ]:
# !pip install quantstats
# !pip install scikit-learn
# !pip install torch
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git
#!pip install vmdpy

In [2]:
import logging
logging.getLogger('matplotlib.font_manager').disabled = True
import torch

import numpy as np

from sklearn.preprocessing import MaxAbsScaler
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import GroupByScaler
from finrl.meta.env_portfolio_optimization.env_portfolio_optimization import PortfolioOptimizationEnv
from finrl.agents.portfolio_optimization.models import DRLAgent
from finrl.agents.portfolio_optimization.architectures import EIIE

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [3]:
TEST_SET = [
    "AAPL", "CVX", "GS", "JNJ",
    "JPM", "MSFT", "PFE", "PG",
    "UL", "XOM"
]
START_DATE = '2000-01-01'
END_DATE = '2023-12-31'
print(len(TEST_SET))

portfolio_raw_df = YahooDownloader(start_date = START_DATE,
                                end_date = END_DATE,
                                ticker_list = TEST_SET).fetch_data()
portfolio_raw_df
portfolio_raw_df.groupby("tic").count()
portfolio_norm_df = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(portfolio_raw_df)
portfolio_norm_df
df_portfolio = portfolio_norm_df[["date", "tic", "close", "high", "low"]]

df_portfolio_train = df_portfolio[(df_portfolio["date"] >= START_DATE) & (df_portfolio["date"] < "2020-12-31")]
df_portfolio_2021 = df_portfolio[(df_portfolio["date"] >= "2021-01-01") & (df_portfolio["date"] < "2021-12-31")]
df_portfolio_2022 = df_portfolio[(df_portfolio["date"] >= "2022-01-01") & (df_portfolio["date"] < "2022-12-31")]
df_portfolio_2023 = df_portfolio[(df_portfolio["date"] >= "2023-01-01") & (df_portfolio["date"] < END_DATE)]

10


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (60370, 8)


/Users/ethan/Documents/GitHub/DSE4212/venv/lib/python3.10/site-packages/finrl/meta/preprocessor/preprocessors.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.07219385 0.06903832 0.1048719  ... 0.00647939 0.00458792 0.00574385]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[select_mask, self.columns] = self.scalers[value].transform(
/Users/ethan/Documents/GitHub/DSE4212/venv/lib/python3.10/site-packages/finrl/meta/preprocessor/preprocessors.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.   0.25 0.5  ... 0.5  0.75 1.  ]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  X.loc[select_mask, self.columns] = self.scalers[value].transform(


In [4]:
import pandas as pd
from vmdpy import VMD

def apply_vmd_to_ticker(df, alpha=5000, tau=0, K=3, DC=0, init=1, tol=1e-7):
    """Applies VMD to the 'close' price for each ticker in the dataframe
    and handles any length mismatch issues."""
    vmd_features = []
    
    # Apply VMD per ticker
    for tic in df['tic'].unique():
        tic_df = df[df['tic'] == tic].copy()  # Filter data for the current ticker
        close_prices = tic_df['close'].to_numpy()  # Extract close prices
        
        # Perform VMD on the close prices
        u, _, _ = VMD(close_prices, alpha, tau, K, DC, init, tol)
        
        # Check if the lengths match, and if not, align them
        if u.shape[1] != len(tic_df):
            print(f"Length mismatch for {tic}: VMD output has {u.shape[1]} points, expected {len(tic_df)}")
            min_len = min(u.shape[1], len(tic_df))
            tic_df = tic_df.iloc[:min_len]  # Truncate the dataframe to match VMD output length
            u = u[:, :min_len]  # Truncate VMD output if necessary
        
        # Assign VMD modes to columns
        for k in range(K):
            tic_df[f'vmd_mode_{k}'] = u[k, :]
        
        # Append the dataframe with VMD modes
        vmd_features.append(tic_df)
    
    # Concatenate all VMD modes into one dataframe
    vmd_df = pd.concat(vmd_features, axis=0)
    
    return vmd_df

# Apply VMD to the training data
df_portfolio_train_vmd = apply_vmd_to_ticker(df_portfolio_train)
df_portfolio_2021_vmd = apply_vmd_to_ticker(df_portfolio_2021)
df_portfolio_2022_vmd = apply_vmd_to_ticker(df_portfolio_2022)
df_portfolio_2023_vmd = apply_vmd_to_ticker(df_portfolio_2023)


Length mismatch for AAPL: VMD output has 5282 points, expected 5283
Length mismatch for CVX: VMD output has 5282 points, expected 5283
Length mismatch for GS: VMD output has 5282 points, expected 5283
Length mismatch for JNJ: VMD output has 5282 points, expected 5283
Length mismatch for JPM: VMD output has 5282 points, expected 5283
Length mismatch for MSFT: VMD output has 5282 points, expected 5283
Length mismatch for PFE: VMD output has 5282 points, expected 5283
Length mismatch for PG: VMD output has 5282 points, expected 5283
Length mismatch for UL: VMD output has 5282 points, expected 5283
Length mismatch for XOM: VMD output has 5282 points, expected 5283
Length mismatch for AAPL: VMD output has 250 points, expected 251
Length mismatch for CVX: VMD output has 250 points, expected 251
Length mismatch for GS: VMD output has 250 points, expected 251
Length mismatch for JNJ: VMD output has 250 points, expected 251
Length mismatch for JPM: VMD output has 250 points, expected 251
Length

In [5]:
environment_vmd = PortfolioOptimizationEnv(
    df=df_portfolio_train_vmd,
    initial_amount=100000,
    comission_fee_pct=0.0025,
    time_window=50,
    features=["close", "high", "low", "vmd_mode_0", "vmd_mode_1", "vmd_mode_2"],  # Add VMD modes as features
    normalize_df=None
)

# set PolicyGradient parameters
model_kwargs = {
    "lr": 0.01,
    "policy": EIIE,
}

# here, we can set EIIE's parameters
policy_kwargs = {
    "k_size": 3,
    "time_window": 50,
    "initial_features": 6
}

model = DRLAgent(environment_vmd).get_model("pg", device, model_kwargs, policy_kwargs)

In [6]:
DRLAgent.train_model(model, episodes=50)


  0%|          | 0/50 [00:00<?, ?it/s]

Initial portfolio value:100000
Final portfolio value: 833287.5625
Final accumulative portfolio value: 8.332875625
Maximum DrawDown: -0.42290616943274806
Sharpe ratio: 0.6487505494239127


  2%|▏         | 1/50 [00:18<15:29, 18.97s/it]

Initial portfolio value:100000
Final portfolio value: 1029413.125
Final accumulative portfolio value: 10.29413125
Maximum DrawDown: -0.4296705430260155
Sharpe ratio: 0.6907156301803709


  4%|▍         | 2/50 [00:37<15:06, 18.89s/it]

...

 96%|█████████▌| 48/50 [15:18<00:37, 18.88s/it]

Initial portfolio value:100000
Final portfolio value: 2.2661273950879744e+16
Final accumulative portfolio value: 226612739508.79745
Maximum DrawDown: -0.1730117826637243
Sharpe ratio: 4.188699140797265


 98%|█████████▊| 49/50 [15:36<00:18, 18.81s/it]

Initial portfolio value:100000
Final portfolio value: 3.3877430720528384e+16
Final accumulative portfolio value: 338774307205.2838
Maximum DrawDown: -0.17515694676074078
Sharpe ratio: 4.213258477193621


100%|██████████| 50/50 [15:56<00:00, 19.12s/it]


In [7]:
torch.save(model.train_policy.state_dict(), "policy_with_VMD.pt")

Evaluation

In [10]:
environment_2021 = PortfolioOptimizationEnv(
    df_portfolio_2021_vmd,
    initial_amount=100000,
    comission_fee_pct=0.0025,
    time_window=50,
    features=["close", "high", "low", "vmd_mode_0", "vmd_mode_1", "vmd_mode_2"],
    normalize_df=None
)

environment_2022 = PortfolioOptimizationEnv(
    df_portfolio_2022_vmd,
    initial_amount=100000,
    comission_fee_pct=0.0025,
    time_window=50,
    features=["close", "high", "low", "vmd_mode_0", "vmd_mode_1", "vmd_mode_2"],
    normalize_df=None
)

environment_2023 = PortfolioOptimizationEnv(
    df_portfolio_2023_vmd,
    initial_amount=100000,
    comission_fee_pct=0.0025,
    time_window=50,
    features=["close", "high", "low", "vmd_mode_0", "vmd_mode_1", "vmd_mode_2"],
    normalize_df=None
)

EIIE_results = {
    "train": {},
    "2021": {},
    "2022": {},
    "2023": {}
}

# instantiate an architecture with the same arguments used in training
# and load with load_state_dict.
policy = EIIE(time_window=50,initial_features=6, device=device)
policy.load_state_dict(torch.load("policy_with_VMD.pt"))
environment_vmd.reset()
DRLAgent.DRL_validation(model, environment_vmd, policy=policy)
EIIE_results["train"]["value"] = environment_vmd._asset_memory["final"]
# 2021
DRLAgent.DRL_validation(model, environment_2021, policy=policy)
EIIE_results["2021"]["value"] = environment_2021._asset_memory["final"]

# 2022
DRLAgent.DRL_validation(model, environment_2022, policy=policy)
EIIE_results["2022"]["value"] = environment_2022._asset_memory["final"]

# 2023
DRLAgent.DRL_validation(model, environment_2023, policy=policy)
EIIE_results["2023"]["value"] = environment_2023._asset_memory["final"]

#uniform buy n hold
UBAH_results = {
    "train": {},
    "2021": {},
    "2022": {},
    "2023": {}
}

PORTFOLIO_SIZE = len(TEST_SET)

# train period
terminated = False
environment_vmd.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_vmd.step(action)
UBAH_results["train"]["value"] = environment_vmd._asset_memory["final"]

# 2021
terminated = False
environment_2021.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_2021.step(action)
UBAH_results["2021"]["value"] = environment_2021._asset_memory["final"]

# 2022
terminated = False
environment_2022.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_2022.step(action)
UBAH_results["2022"]["value"] = environment_2022._asset_memory["final"]

# 2023
terminated = False
environment_2023.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_2023.step(action)
UBAH_results["2023"]["value"] = environment_2023._asset_memory["final"]

/var/folders/lk/jtg57k7555qg6vh9z23sx18m0000gn/T/ipykernel_71880/4118930828.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  policy.load_state_dict(torch.load("policy_wi

Initial portfolio value:100000
Final portfolio value: 1.1892392927952896e+16
Final accumulative portfolio value: 118923929279.52896
Maximum DrawDown: -0.1729974244690209
Sharpe ratio: 4.0288531868574795
Initial portfolio value:100000
Final portfolio value: 144675.546875
Final accumulative portfolio value: 1.44675546875
Maximum DrawDown: -0.14204695888082508
Sharpe ratio: 1.907739808357341
Initial portfolio value:100000
Final portfolio value: 207220.8125
Final accumulative portfolio value: 2.072208125
Maximum DrawDown: -0.17429665635838065
Sharpe ratio: 2.9113116990931487
Initial portfolio value:100000
Final portfolio value: 161391.671875
Final accumulative portfolio value: 1.61391671875
Maximum DrawDown: -0.0555481684213911
Sharpe ratio: 2.747510215294065


In [1]:
import matplotlib.pyplot as plt
%matplotlib inline 

plt.plot(UBAH_results["train"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["train"]["value"], label="EIIE")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in training period")
plt.legend()

plt.show()

plt.plot(UBAH_results["2021"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["2021"]["value"], label="EIIE")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in 2021")
plt.legend()

plt.show()

plt.plot(UBAH_results["2022"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["2022"]["value"], label="EIIE")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in 2022")
plt.legend()

plt.show()

plt.plot(UBAH_results["2023"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["2023"]["value"], label="EIIE")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in 2023")
plt.legend()

plt.show()

NameError: name 'UBAH_results' is not defined